In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
pd.set_option('future.no_silent_downcasting', True)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. filename

In [4]:
out_dir = cfg.config['out_dir']
outcome_filename = f'{out_dir}/out-merged-normalized/20-10-outcome.csv'

df = pd.read_csv(outcome_filename)

In [5]:
is_death_before_followup = df['deathBeforeFollowup'].copy().fillna(False)

In [6]:
df['disabilityLevelDeath'] = df['disabilityLevelDeath4Category'].copy()
df.loc[is_death_before_followup, 'disabilityLevelDeath'] = 'death'

In [7]:
columns = ['uniqueID', 'disabilityLevelDeath4Category', 'deathBeforeFollowup', 'disabilityLevelDeath']
df[columns]

,uniqueID,disabilityLevelDeath4Category,deathBeforeFollowup,disabilityLevelDeath
0,03:LH087,severe,True,death
1,03:LH088,severe,False,severe
2,03:LH092,normal,False,normal
3,03:LH094,mild,False,mild
4,03:LH098,normal,False,normal
...,...,...,...,...
527,31:OC3411,NaN,False,NaN
528,31:OC3441,normal,False,normal
529,31:OC3471,severe,True,death
530,31:OC3551,normal,False,normal


In [8]:
columns = ['_study', 'center', 'subjectID', 'uniqueID', 'followupCenter', 'disabilityLevelDeath']

df_out = df[columns]
for idx in ['center', 'followupCenter']:
    df_out[idx] = df_out[idx].astype('O')
    df_out.loc[:, idx] = df_out[idx].apply(COMBINE_harmonizer.to_center)

out_filename = f'{out_dir}/out-merged-normalized/20-10_1-disability-level-death.csv'
df_out.to_csv(out_filename, index=False)